In [ ]:
import argparse  # for parsing command line arguments
import yaml  # for parsing the configuration file
import os  # for creating folders
import signal  # for handling signals
import sqlite3  # for saving the data
import threading  # for running multiple threads
import time  # for performance measurement
import datetime  # for storage addressing
import pytz  # "python time zone" library for precision time zone handling
import typing  # for type hints
from typing import Callable, Any

from paho.mqtt import client as mqtt
import pathlib
import json

In [ ]:
q_stream_path = pathlib.Path("C:\\git\\attic_data\\everything")

In [ ]:
timestamp_iso_string_inner = datetime.datetime.now(tz=pytz.UTC).isoformat(timespec='microseconds')
timestamp_iso_string_inner

In [ ]:
folder_names_inner = timestamp_iso_string_inner.split('T')[0].split('-')
"/".join(folder_names_inner)

In [ ]:
import pathlib
import re
"/".join(folder_names_inner)
pathlib.Path().mkdir(parents=True, exist_ok=True)

In [ ]:
datetime.datetime.now(tz=pytz.UTC)

In [ ]:
def datetime_to_safestring(timestamp: datetime.datetime) -> str:
    unsafestring = datetime.datetime.now(tz=pytz.UTC).isoformat(timespec='microseconds')
    safestring = re.sub(r'[/:*?"<>|.]', '_', unsafestring)
    return safestring

def safestring_to_datetime(safestring: str) -> datetime.datetime:
    unsafestring = re.sub(r'[_]', ':', safestring)
    return datetime.datetime.fromisoformat(unsafestring)

def test_datetime_safestring_conversion():
    log = lambda x: None
    timestamp = datetime.datetime.now(tz=pytz.UTC)
    log(timestamp)
    safestring = datetime_to_safestring(timestamp)
    log(safestring)
    timestamp_recovered = safestring_to_datetime(safestring)
    log(timestamp_recovered)
    timedelta = timestamp_recovered - timestamp
    timedelta_seconds = timedelta.total_seconds()
    assert(timedelta_seconds < 1e-2)
    log(timedelta_seconds)
    
test_datetime_safestring_conversion()

In [ ]:
timestamp_iso_string_inner = datetime.datetime.now(tz=pytz.UTC).isoformat(timespec='microseconds')
print(timestamp_iso_string_inner)
file_name = f'{timestamp_iso_string_inner}'
safe_filename = re.sub(r'[/:*?"<>|.]', '_', file_name)
print(safe_filename)
timestamp_str_recovered = re.sub(r'[_]', ':', safe_filename)
print(timestamp_str_recovered)
datetime_back_from_safefilename = datetime.datetime.fromisoformat(timestamp_str_recovered)
print(datetime_back_from_safefilename)
# prepare folder name
# folder per day should be sufficient for this application
folder_names_inner = timestamp_iso_string_inner.split('T')[0].split('-')
full_folder_name = pathlib.Path(q_stream_path, *folder_names_inner)
full_folder_name

In [ ]:
type(datetime.datetime.now(tz=pytz.UTC))

In [ ]:
bsqlite_db_path = full_folder_name.joinpath(safe_filename)
sqlite_db_path

In [ ]:
# get the files in that path
files = [x for x in os.listdir(full_folder_name) if x.endswith('.sqlite')]

In [ ]:
files
# open the latest sqlite file with pandas
import pandas as pd
import sqlite3
conn = sqlite3.connect(full_folder_name.joinpath(files[-1]))
df = pd.read_sql_query("SELECT * FROM data", conn)
# select only where topic is "Temperatures/temperatures"
df_temperatures = df[df.topic == "Temperatures/temperatures"].copy()
# decode json payload into columns
df_temperatures.loc[:,'json'] = df_temperatures.payload.apply(lambda x: json.loads(x.decode('utf-8')))
df_temperatures.drop(columns=['payload'], inplace=True)
df_temperatures = pd.json_normalize(df_temperatures.json)
df_temperatures_expanded = df_temperatures.temperatures.apply(pd.Series)
df_temperatures = pd.concat([df_temperatures.drop(columns='temperatures') , df_temperatures_expanded], axis=1)
df_temperatures['timestamp']=df_temperatures.timestamp.apply(lambda x: datetime.datetime.fromisoformat(x))
# df_temperatures_expanded
df_temperatures

In [ ]:
from matplotlib import pyplot as plt
hf = plt.figure(figsize=[9,6], dpi=100)
href = plt.plot(df_temperatures.timestamp, df_temperatures[0], label='ref')
h1 = plt.plot(df_temperatures.timestamp, df_temperatures[1], label='ch1')
h1 = plt.plot(df_temperatures.timestamp, df_temperatures[2], label='ch2')
plt.grid()
plt.show()

In [ ]:
datetime.daatetime.fromisoformat(df_temperatures.timestamp[0])

In [ ]:
# list the tables in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())